In [1]:
# To hold the code of a blank transformer model
import sys
sys.path.append('/workspace/fourth_year_project/MusicGen')
#print(sys.path)

In [2]:
from MyAudioDataset import MyAudioDataset

In [9]:
import torch
from torch import nn


In [10]:

class AudioTransformer(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward):
        super(AudioTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers, dim_feedforward)
        self.input_encoding = nn.Linear(1, d_model)  # input of mono audio
        #self.angle_encoding = nn.Linear(6, d_model)  # add this once 90 works. 
        self.output_decoding = nn.Linear(d_model, 2)  # Decoding back to stereo audio

    def forward(self, audio, angle):
        # audio: (batch_size, 1, seq_length)
        # angle: (batch_size, 6)  # one-hot encoded angle
        audio = self.input_encoding(audio)

        #angle = self.angle_encoding(angle)  # Process one-hot encoded angle
        #angle = angle.unsqueeze(1).repeat(1, audio.size(2), 1)  # Repeat angle for each time step
        #x = audio + angle  # Combine audio and angle

        x = self.transformer(x.permute(2, 0, 1))  # Transformer expects input in the form (seq_length, batch_size, d_model)
        x = x.permute(1, 2, 0)  # Bring it back to (batch_size, d_model, seq_length)
        x = self.output_decoding(x)
        return x.view(-1, 2, 30*44100)  # Reshape to stereo audio
    
    def train(self, dataset, batch_size, epochs, lr):
        # dataset: MyAudioDataset object
        # batch_size: int
        # epochs: int
        # lr: float
        #loss_fn = nn.MSELoss()
        loss_fn = nn.CosineEmbeddingLoss
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            for i, (target, orig, angle, sr) in enumerate(train_loader):
                optimizer.zero_grad()
                output = self(orig, angle)
                loss = loss_fn(output, target)
                loss.backward()
                optimizer.step()
                print(f"Epoch {epoch}, batch {i}, loss {loss.item()}")

            torch.save(self.state_dict(), f"model_{epoch}.pth")
            print(f"Saved model_{epoch}.pth")

        print("Finished Training")

In [11]:
mydataset = MyAudioDataset('/workspace/small_model_data2', 'recording_01_')

In [12]:
myTransformer = AudioTransformer(1, 1, 1, 1)
myTransformer.train(mydataset, 1, 1, 1)

/workspace/venv_work/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple

In [4]:
a = mydataset.get_sample(0)

In [5]:
a['target'].shape

torch.Size([2, 960000])

In [6]:
a['original'].shape

torch.Size([1, 960000])

In [7]:
a

{'target': tensor([[-0.0043,  0.0054,  0.0142,  ..., -0.0016,  0.0088,  0.0170],
         [-0.0078, -0.0058, -0.0029,  ...,  0.0104,  0.0040, -0.0041]]),
 'original': tensor([[ 0.0902,  0.0749,  0.0478,  ..., -0.0333, -0.0623, -0.0783]]),
 'label': '90',
 'sr': 32000}